# Image Recognizer Demo with ScienceOps
Deploying an image tagging model using neural networks + Python.

## VGG16 model for Keras

[Paper here](https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3)

This is the [Keras](http://keras.io/) model of the 16-layer network used by the VGG team in the ILSVRC-2014 competition.

It has been obtained by directly converting the [Caffe model](https://gist.github.com/ksimonyan/211839e770f7b538e2d8#file-readme-md) provived by the authors.

Details about the network architecture can be found in the following arXiv paper:

    Very Deep Convolutional Networks for Large-Scale Image Recognition
    K. Simonyan, A. Zisserman
    arXiv:1409.1556
    

In the paper, the VGG-16 model is denoted as configuration `D`. It achieves 7.5% top-5 error on ILSVRC-2012-val, 7.4% top-5 error on ILSVRC-2012-test.

Please cite the paper if you use the models.

### Contents:

model and usage demo: see `vgg-16_keras.py`

weights: [vgg16_weights.h5](https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing)

## Before you start

Download the weights from this link: https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view?usp=sharing and put them in a directory called `data/`.

In [1]:
!tree data

data
├── female_blog_list.txt
├── intro_to_ann.csv
├── male_blog_list.txt
├── mnist.pkl.gz
├── pig.jpg
├── rt-polarity.neg
├── rt-polarity.pos
└── vgg16_weights.h5

0 directories, 8 files


### Setting things up

In [2]:
##########
# basics #
##########

import warnings
warnings.filterwarnings('ignore')
import collections
import datetime
import glob
import hashlib
import itertools
import math
import operator
import os
import pickle
import random
import re
import string
import sys
import time

###########
# science #
###########

import scipy as sp
import numpy as np
import pandas as pd
rseed = random.seed(42)
np.random.seed(rseed)

######
# ml #
######

import theano as thno
import keras as krs
import tensorflow as tf

###################
# sklearn tooling #
###################

from sklearn import decomposition
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection
from sklearn import grid_search
from sklearn import pipeline
from sklearn import feature_selection

#################
# visualization #
#################

# matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
mpl.rcParams['figure.figsize']=(12.0,4.0)
%matplotlib inline

############
# sys info #
############

%reload_ext watermark
%watermark -a "Ken Cavagnolo" -n -u -v -m -h -g -p numpy,scipy,pandas,sklearn,theano,keras,tensorflow,\
matplotlib,plotly

Using TensorFlow backend.
/usr/local/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Ken Cavagnolo 
last updated: Tue Nov 08 2016 

CPython 2.7.12
IPython 5.1.0

numpy 1.11.2
scipy 0.18.1
pandas 0.19.0
sklearn 0.18
theano 0.8.2
keras 1.1.0
tensorflow 0.10.0rc0
matplotlib 1.5.3
plotly 1.12.9

compiler   : GCC 4.4.7 20120313 (Red Hat 4.4.7-1)
system     : Linux
release    : 4.4.0-42-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
host name  : ubuntu
Git hash   : ac1755e7e0c839e42c71d4577b9a246ce92e1477


In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import cv2
import numpy as np
import pprint as pp

Using TensorFlow backend.


ImportError: No module named cv2

In [ ]:
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

### Munging our data

In [ ]:
labels = [" ".join(row.split(' ')[1:]) for row in open("./labels.txt").read().strip().split('\n')]
im = cv2.resize(cv2.imread('images/pig.jpg'), (224, 224)).astype(np.float32)
# im = cv2.resize(cv2.imread('images/pickup-truck.jpg'), (224, 224)).astype(np.float32)
# im = cv2.resize(cv2.imread('images/cat.jpg'), (224, 224)).astype(np.float32)
im[:,:,0] -= 103.939
im[:,:,1] -= 116.779
im[:,:,2] -= 123.68
im = im.transpose((2,0,1))
im = np.expand_dims(im, axis=0)

### Building the model

In [ ]:
# Test pretrained model
model = VGG_16('data/vgg16_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy')

### Making a prediction

In [ ]:
out = model.predict(im)
pred = dict(zip(labels, model.predict_proba(im)[0]))
best_guess = labels[np.argmax(out)]
print "It's a %s" % best_guess
print { "guess": "It's a %s" % best_guess}

![](images/pig.jpg)

<center><h3>`{"guess": "hog, pig, grunter, squealer, Sus scrofa"}`</h3></center>

### Deploying to ScienceOps

In [ ]:
from yhat import Yhat, YhatModel, preprocess
from PIL import Image
from StringIO import StringIO
import base64

class ImageRecognizer(YhatModel):
    REQUIREMENTS = [
        "opencv"
    ]
    @preprocess(in_type=dict, out_type=dict)
    def execute(self, data):
        img64 = data['image64']
        binaryimg = base64.decodestring(img64)
        pilImage = Image.open(StringIO(binaryimg))
        image = np.array(pilImage)
        resized_image = cv2.resize(image, (224, 224)).astype(np.float32)
        resized_image[:,:,0] -= 103.939
        resized_image[:,:,1] -= 116.779
        resized_image[:,:,2] -= 123.68
        resized_image = resized_image.transpose((2,0,1))
        resized_image = np.expand_dims(resized_image, axis=0)

        out = model.predict(resized_image)
        pred = dict(zip(labels, model.predict_proba(im)[0]))
        best_guess = labels[np.argmax(out)]
        print "It's a %s" % best_guess
        return { "guess": best_guess }


testdata = {
    "image64": open('./test-image.base64', 'rb').read()
}
print ImageRecognizer().execute(testdata)

In [ ]:
yh = Yhat(USERNAME, APIKEY, URL)
yh.deploy("ImageRecognizer", ImageRecognizer, globals(), True)